In [108]:
# 모두닥 크롤링 모듈

In [109]:
import sys

sys.version

'3.8.13 (default, Mar 28 2022, 06:59:08) [MSC v.1916 64 bit (AMD64)]'

In [110]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [202]:
import pandas as pd    # csv
from selenium.webdriver.chrome.options import *
from bs4 import *
import time
import re
import base64    # encoder
# webdriver = 웹페이지를 제어하기 위한 모듈 >> 웹 그자체가 됨!
from selenium import webdriver as wb
# Keys = 컴퓨터의 키보드와 같은 역할 모듈, 컴퓨터가 쓸 키보드 
from selenium.webdriver.common.keys import Keys

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [204]:
# 로그인
def login(id, password):
    driver.find_element('xpath', '/html/body/nav/div/div/ul[2]/ul/li[3]/a').click() #id 속성으로 접근

    user_email = driver.find_element('xpath', '/html/body/div[12]/div/div/div[3]/form/input[1]')
    user_email.send_keys(id)
    user_password = driver.find_element('xpath', '/html/body/div[12]/div/div/div[3]/form/input[2]')
    user_password.send_keys(password)

    # 로그인 버튼 클릭
    driver.find_element('xpath', '/html/body/div[12]/div/div/div[3]/form/button').click()
    time.sleep(3)

    
# 크롤링 하려는 병원 이동
def move_to_hospital(url):
    driver.get(url)
    time.sleep(1)

    
# 병원의 리뷰 페이지 수 계산
def count_page():
    num_review = driver.find_element('xpath', '/html/body/div[3]/div/div[2]/div[2]/div/div[1]/div[1]').text[7:]
    num_review = int(num_review)

    num_page = (num_review // 10) + 1

    return num_page


# 텍스트 전처리
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]' , '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]','', cleaned_text)
    cleaned_text = re.sub('[\n]', ' ', cleaned_text)
    
    return cleaned_text


# 병원 페이지로 이동
def move_to_hospital(hospital):
    search_box = driver.find_element('xpath', '/html/body/div[2]/div/div[3]/form/div/div/span/input[2]')
    search_box.send_keys(hospital)
    driver.find_element('xpath', '/html/body/div[2]/div/div[3]/form/div/div/div/button/img').click()
    time.sleep(2)

    for i in range(10):
        try:
            element = driver.find_element('xpath', f'/html/body/div[3]/div/div/div[1]/div[3]/div[{i}]/a/div/div[2]/div[2]/div[1]/div[1]/div[1]')

            if element.text[:3] == hospital[:3]:
                element.click()
                time.sleep(2)
                break

        except:
            pass

In [189]:
# 모두닥 페이지 접속 후 로그인

In [205]:
# 모두닥 페이지 url
modoodoc_url = "https://www.modoodoc.com/"

## 크롬 드라이버 실행
#  wb.Chrome() = 컴퓨터가 크롬창을 씀
driver = wb.Chrome()

# 모두닥 페이지 이동
driver.get(modoodoc_url)

# 로그인
id = 'random1234@random.com'
password = 'random1234'
login(id, password)

In [206]:
# 읽어올 엑셀 파일 지정
filename = 'temp.xlsx'

# 엑셀 파일 읽어 오기
data = pd.read_excel(filename, engine='openpyxl',  )
hospital_list = data.병원이름.to_list()

In [ ]:
# 크롤링 후 리스트 삽입

data = []
data_unit = []

for index in range(len(hospital_list)):
    # 병원 이름 검색 통해 접근(리스트 불러와서 넣는 것으로 변경 예정)
    hospital = hospital_list[index]
    move_to_hospital(hospital)
    
    current_url = driver.current_url
    num_page = count_page()
    
    for page in range(1, num_page+1):
        url = current_url + f"?page={page}"
        driver.get(url)
        time.sleep(3)

        for j in range(2, 16):
            try:
                # 병원 이름 크롤링
                hospital = driver.find_element('xpath', '/html/body/div[3]/div/div[2]/div[1]/h1').text

                # 받은 치료 이름 크롤링
                treatment = driver.find_element('xpath', f'/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[{j}]/div/div[2]/div[2]').text


                # 받은 치료 명시 안된 부분 처리
                if treatment[0] != '받':
                    continue
                treatment = treatment[8:]

                # 리뷰 내용 크롤링
                review = driver.find_element('xpath', f'/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[{j}]/div/div[2]/div[3]').text
                # 리뷰 텍스트 전처리
                review = clean_text(review)

                # 크롤링 데이터 리스트 삽입
                data_unit = [hospital, treatment, review]
                data.append(data_unit)

            except:
                pass
    
    
    
    driver.get(modoodoc_url)
    time.sleep(3)

In [198]:
# 데이터 프레임 생성 및 데이터 입력
df = pd.DataFrame(columns = ['병원 이름', '받은 치료', '리뷰'])

num = len(data)

for i in range(num):
    df.loc[i]=data[i]
   
df

,병원 이름,받은 치료,리뷰
0,프리미어성형외과의원,코성형수술,한줄평 거울 부시고싶은 코 수술 전에 환자가 원하는 모양에 대한 충분한 커뮤니케이...
1,프리미어성형외과의원,코성형수술,한줄평 거울 부시고싶은 코 수술 전에 환자가 원하는 모양에 대한 충분한 커뮤니케이...
2,프리미어성형외과의원,코성형수술,한줄평 거울 부시고싶은 코 수술 전에 환자가 원하는 모양에 대한 충분한 커뮤니케이...
3,프리미어성형외과의원,코성형수술,한줄평 거울 부시고싶은 코 수술 전에 환자가 원하는 모양에 대한 충분한 커뮤니케이...
4,프리미어성형외과의원,코성형수술,한줄평 거울 부시고싶은 코 수술 전에 환자가 원하는 모양에 대한 충분한 커뮤니케이...
...,...,...,...
135,오가나셀피부과의원,점/비립종/쥐젖 제거 (레이저),점 크기가 커서 5회정도 제거가 필요했습니다 오가나 원장님 말고도 여성 부원장님 계...
136,오가나셀피부과의원,점/비립종/쥐젖 제거 (레이저),점 크기가 커서 5회정도 제거가 필요했습니다 오가나 원장님 말고도 여성 부원장님 계...
137,오가나셀피부과의원,점/비립종/쥐젖 제거 (레이저),점 크기가 커서 5회정도 제거가 필요했습니다 오가나 원장님 말고도 여성 부원장님 계...
138,오가나셀피부과의원,점/비립종/쥐젖 제거 (레이저),점 크기가 커서 5회정도 제거가 필요했습니다 오가나 원장님 말고도 여성 부원장님 계...


In [199]:
# 엑셀 저장
df.to_excel('크롤링결과.xlsx')

In [117]:
# 버전 체크

print(webdriver.__version__)

4.3.0


In [118]:
# # 병원 이름 크롤링
# hospital = driver.find_element('xpath', '/html/body/div[3]/div/div[2]/div[1]/h1').text

# # 받은 치료 이름 크롤링
# treatment = driver.find_element('xpath', '/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[5]/div/div[2]/div[2]').text[8:]

# # 리뷰 내용 크롤링
# review = driver.find_element('xpath', '/html/body/div[3]/div/div[4]/div[1]/div[4]/div[5]/div[5]/div/div[2]/div[3]').text

# # 크롤링 데이터 리스트 삽입
# data_unit = [hospital, treatment, review]
# data.append(data_unit)
# data

In [119]:
# parsing을 위한 함수를 생성

In [120]:
# def remove_html_tags(data):
#     p = re.compile(r'<.*?>')
#     return p.sub(' ', str(data))

# def get_crawl(URL):
#     response = driver.get(URL)
#     html = driver.page_source
#     soup7 = BeautifulSoup(html, 'html.parser')
#     ex_id_divs = soup7.find('div', {'id': 'view_content'})
#     crawl_data = remove_html_tags(ex_id_divs)
#     return crawl_data

In [121]:
# # 리뷰 페이지 이동
# num_page = count_page()

# current_url = driver.current_url

# for i in range(2, num_page+1):
#     url = current_url + f"?page={i}"
#     driver.get(url)
#     time.sleep(1)

In [201]:
# # 병원 이름 검색 통해 접근(리스트 불러와서 넣는 것으로 변경 예정)
# hospital = input()
# search_box = driver.find_element('xpath', '/html/body/div[2]/div/div[3]/form/div/div/span/input[2]')
# search_box.send_keys(hospital)
# driver.find_element('xpath', '/html/body/div[2]/div/div[3]/form/div/div/div/button/img').click()
# time.sleep(1)

# for i in range(10):
#     try:
#         element = driver.find_element('xpath', f'/html/body/div[3]/div/div/div[1]/div[3]/div[{i}]/a/div/div[2]/div[2]/div[1]/div[1]/div[1]')
    
#         if element.text[:2] == hospital[:2]:
#             element.click()
#             break
            
#     except:
#         pass
        
# time.sleep(1)

디에이성형외과
